In [10]:
import urllib.request as url
import ssl
from multiprocessing import Pool
from bs4 import BeautifulSoup as BS
import time
import os

# Obter atas do Copom
### Repositório do Github: https://github.com/paulodt/TCC-PauloDeTarso

## Código para transformar os pdfs em texto

Criado a partir da modificação de:

http://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library

In [11]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

def convert_pdf(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    process_pdf(rsrcmgr, device, fp)
    fp.close()
    device.close()

    str = retstr.getvalue()
    retstr.close()
    return str

## Baixar as atas do COPOM

Para ignorar a verificação de certificados, me baseio em:
http://stackoverflow.com/questions/36600583/python-3-urllib-ignore-ssl-certificate-verification

In [12]:
#Ignorar a verificação de certificados no "urlopen"
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

Aqui, crio uma função para tratar de cada ata e poder paralelizar o processo.

Se a ata estiver em pdf, utilizo o programa "convert_pdf" acima para transformar em um txt. Se for um html, transformo em um txt diretamente.

In [16]:
#Parâmetros
total_atas = 207
atas = range(total_atas)
path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/"
path_atas = path + "/Atas/"

for n_ata in atas:
    try:
        #Progresso
        print("Progresso: " + str(round(100*n_ata/total_atas)) + "%",end="\r")
        
        #Parâmetros
        ata = str(n_ata)
        link = "https://www.bcb.gov.br/?COPOM" + ata
        filename = "COPOM_" + ata

        #Acessar o site e obter o tipo (html ou PDF)
        site = url.urlopen(link,context=ctx)
        content_type = dict(site.getheaders())["Content-Type"]

        #Checar se é um PDF ou um HTML
        if "pdf" in content_type:

            #Baixar o arquivo
            with open(path + filename + ".pdf","wb") as file_in:
                file_in.write(site.read())
            file_in.close()

            full_text = convert_pdf(filename + ".pdf")

            #Escrever no arquivo txt
            file_out = open(path_atas + filename + ".txt","w")
            file_out.write(full_text[full_text.index("Data:"):])
            file_out.close()

            #Remover o arquivo original
            os.remove(path + filename + ".pdf")

        else:
            soup = BS(site.read(),"lxml")
            full_text = "\n".join(soup.findAll(text=True))

            #Escrever no arquivo txt
            file_out = open(path_atas + filename + ".txt","w")
            file_out.write(full_text[full_text.index("Data:"):])
            file_out.close()
    
    except:
        continue